In [ ]:
!pip install bs4
!pip install openpyxl

In [ ]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json

In [ ]:
class rbc_parser:
    def __init__(self):
        pass

    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        url = 'https://www.rbc.ru/search/ajax/?' +\
        'project={0}&'.format(param_dict['project']) +\
        'category={0}&'.format(param_dict['category']) +\
        'dateFrom={0}&'.format(param_dict['dateFrom']) +\
        'dateTo={0}&'.format(param_dict['dateTo']) +\
        'page={0}&'.format(param_dict['page']) +\
        'query={0}&'.format(param_dict['query']) +\
        'material={0}'.format(param_dict['material'])

        return url


    def _get_search_table(self, param_dict: dict,
                          include_text: bool = True) -> pd.DataFrame:
        url = self._get_url(param_dict)
        try:
            r = rq.get(url)
            search_table = pd.DataFrame(r.json()['items'])
        except json.JSONDecodeError as e:
            print(f"Ошибка: {e}")
        if include_text and not search_table.empty:
            get_text = lambda x: self._get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()

        if 'publish_date_t' in search_table.columns:
            search_table.sort_values('publish_date_t', ignore_index=True)

        return search_table


    def _iterable_load_by_page(self, param_dict):
        param_copy = param_dict.copy()
        results = []

        result = self._get_search_table(param_copy)
        results.append(result)

        while not result.empty:
            param_copy['page'] = str(int(param_copy['page']) + 1)
            result = self._get_search_table(param_copy)
            results.append(result)

        results = pd.concat(results, axis=0, ignore_index=True)

        return results


    def _get_article_data(self, url: str):
        r = rq.get(url)
        soup = bs(r.text, features="lxml")
        div_overview = soup.find('div', {'class': 'article__text__overview'})
        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        else:
            overview = None
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None

        return overview, text

    def get_articles(self,
                     param_dict,
                     time_step = 1,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom
        ###### dateTo
        ###### query - поисковой запрос (ключевое слово), например, РБК
        ###### page - смещение поисковой выдачи (с шагом 20)
        ###### limit
        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%d.%m.%Y')
        dateTo = datetime.strptime(param_copy['dateTo'], '%d.%m.%Y')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo or len(out):
            param_copy['dateTo'] = (dateFrom + time_step).strftime("%d.%m.%Y")
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime("%d.%m.%Y")
            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = pd.concat([out, self._iterable_load_by_page(param_copy)], axis=0, ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime("%d.%m.%Y")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0
            if len(out) > param_dict['limit']:
                break

        if save_excel:
            out.to_excel("rbc_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

In [ ]:
use_parser = "РБК"

query = 'РБК'
project = "rbcnews"
category = "TopRbcRu"
material = ""
dateFrom = '2025-01-01'
dateTo = "2025-04-30"
page = 0

param_dict = {'query'   : query,
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'page'   : str(page),
                  'material': material,
                  'limit': 3000}

print(use_parser, "- param_dict:", param_dict)

РБК - param_dict: {'query': 'РБК', 'project': 'rbcnews', 'category': 'TopRbcRu', 'dateFrom': '01.01.2025', 'dateTo': '30.04.2025', 'page': '0', 'material': '', 'limit': 3000}


In [ ]:
parser = rbc_parser()
table = parser.get_articles(param_dict=param_dict,
                             time_step = 1, # Шаг
                             save_every = 100, # Сохранять чекпойнт каждые 100 шагов
                             save_excel = True) # Сохранить итоговый файл
print(len(table))
table.head()

Parsing articles from 01.01.2025 to 02.01.2025
Parsing articles from 03.01.2025 to 04.01.2025
Parsing articles from 05.01.2025 to 06.01.2025
Parsing articles from 07.01.2025 to 08.01.2025
Parsing articles from 09.01.2025 to 10.01.2025
Parsing articles from 11.01.2025 to 12.01.2025
Parsing articles from 13.01.2025 to 14.01.2025
Parsing articles from 15.01.2025 to 16.01.2025
Parsing articles from 17.01.2025 to 18.01.2025
Parsing articles from 19.01.2025 to 20.01.2025
Parsing articles from 21.01.2025 to 22.01.2025
Finish
3099


,id,project,project_nick,type,category,title,body,publish_date,publish_date_t,fronturl,picture,badge,pay_option,data,_score,overview,text
0,6776f2db9a7947e147db3050,РБК,rbcnews,article,Политика,Подозреваемый в подрыве Cybertruck у отеля Тра...,Водитель взорвавшегося в Лас-Вегасе Cybertruck...,2025-01-02T23:35:42+03:00,1735850142,https://www.rbc.ru/politics/02/01/2025/6776f2d...,None,None,free,None,1,Водитель взорвавшегося в Лас-Вегасе Cybertruck...,Водитель взорвавшегося в Лас-Вегасе автомобил...
1,6776e6f89a79471c0fd6ea4b,РБК,rbcnews,short_news,Политика,Наехавший на толпу в Новом Орлеане был членом ИГ,,2025-01-02T23:24:22+03:00,1735849462,https://www.rbc.ru/rbcfreenews/6776e6f89a79471...,None,None,free,None,1,None,"Водитель, 1 января наехавший на толпу в Новом ..."
2,6776eb819a79470a26b9c585,РБК,rbcnews,short_news,Политика,За два часа над Россией уничтожили семь украин...,,2025-01-02T22:54:42+03:00,1735847682,https://www.rbc.ru/rbcfreenews/6776eb819a79470...,None,None,free,None,1,None,С 20:00 до 22:00 российские ПВО уничтожили сем...
3,6776db3c9a794762b5a8ad37,РБК,rbcnews,short_news,Политика,ФБР не нашла связь между терактами 1 января в ...,,2025-01-02T22:22:04+03:00,1735845724,https://www.rbc.ru/rbcfreenews/6776db3c9a79476...,https://s0.rbk.ru/v6_top_pics/media/img/6/27/3...,None,free,None,1,None,Следователи ФБР не нашли связь между терактом ...
4,6776dccb9a79470afdbdb5a0,РБК,rbcnews,article,Политика,Стефанчук сообщил о повреждении здания Рады пр...,Здания Рады в Киеве были повреждены взрывной в...,2025-01-02T22:15:12+03:00,1735845312,https://www.rbc.ru/politics/02/01/2025/6776dcc...,https://s0.rbk.ru/v6_top_pics/media/img/7/37/3...,None,free,None,1,Здания Рады в Киеве были повреждены взрывной в...,В результате взрывов в центре Киева 1 января в...
